<a href="https://colab.research.google.com/github/Jyothiraditya135/Some-Codes/blob/main/Implementing_a_Transformer_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchtext==0.6.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import math
import copy
import spacy

In [ ]:
'''
# Seeding for reproducible results everytime
SEED = 777

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True'''

'\n# Seeding for reproducible results everytime\nSEED = 777\n\nrandom.seed(SEED)\nnp.random.seed(SEED)\ntorch.manual_seed(SEED)\ntorch.cuda.manual_seed(SEED)\ntorch.backends.cudnn.deterministic = True'

In [ ]:

!python -m spacy download en_core_web_sm --quiet
!python -m spacy download de_core_news_sm --quiet

2023-07-23 11:26:34.016168: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2023-07-23 11:26:52.409556: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 55.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [ ]:
spacy_de = spacy.load("de_core_news_sm")
spacy_en = spacy.load("en_core_web_sm")

In [ ]:
def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)]

def tokenize_en(text):
  return [token.text for token in spacy_en.tokenizer(text)]

In [ ]:
DE_TEXT = Field(tokenize=tokenize_de, lower=True, init_token="<sos>", eos_token="<eos>", unk_token="<unk>")
EN_TEXT = Field(tokenize=tokenize_en, lower=True, init_token="<sos>", eos_token="<eos>", unk_token="<unk>")

train_data, valid_data, test_data = Multi30k.splits(exts = (".de", ".en"),
                                                    fields=(DE_TEXT, EN_TEXT))

DE_TEXT.build_vocab(train_data, max_size=10001, min_freq=3)
EN_TEXT.build_vocab(train_data, max_size=10001, min_freq=3)

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:00<00:00, 6.86MB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 1.86MB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 1.66MB/s]


In [ ]:
if "<pad>" in EN_TEXT.vocab.stoi:
  print('Yes')

Yes


In [ ]:
if "<pad>" in DE_TEXT.vocab.stoi:
  print('Yes')

Yes


In [ ]:
len(EN_TEXT.vocab)

4556

In [ ]:
print(min(EN_TEXT.vocab.stoi.values()))
print(max(EN_TEXT.vocab.stoi.values()))

0
4555


In [ ]:
len(DE_TEXT.vocab)

5374

In [ ]:
print(min(DE_TEXT.vocab.stoi.values()))
print(max(DE_TEXT.vocab.stoi.values()))

0
5373


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

train_iter, valid_iter, test_iter = BucketIterator.splits((train_data, valid_data, test_data),
                                                                      batch_size = BATCH_SIZE,
                                                                      sort_within_batch=True,
                                                                      sort_key=lambda x: len(x.src),
                                                                      device = device)

In [ ]:
# Assuming you have an iterator named `train_iter` for your training data
batch = next(iter(train_iter))

# Access tokenized sequences for source and target languages
source_tokens = batch.src
target_tokens = batch.trg

# Print the tokenized sequences for the first batch
print("Source Tokens:")
print(source_tokens)
print("Target Tokens:")
print(target_tokens)


Source Tokens:
tensor([[   2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2],
        [   8,    5,   15,    8,    5,  221,   18,    5,    5,    5,   18,   43,
            5,  325,    8,    8,    5,    5,    5,   18,   18,    5,    8,    5,
           65,    5,    5,    5,    8,    5,    8,   18],
        [  16,  777, 1611,  165,  130,   30,   80, 2859,  385,   32,   30,   54,
           13,   29,   26,  113,  431,   13,   26,   26,  103,   13,   36,   13,
         2120,   13,   66,   32,   36,   66,  364, 5071],
        [  98,  580,   32,  350,   13,   10,   52,  116,   25,   83,   52,   57,
           31, 1400,   16,    9,   68,   11,   11,  330,   80,    7,   22,    7,
          125,   11,   25,  443,   22,   25,   10,   12],
        [  14,  182,   83,   11,   11,   45,   12,    9,    9,   21,    7,   20,
           21, 1703,  20

In [ ]:
print("<pad> token index:", DE_TEXT.vocab.stoi['<pad>'])
print("<sos> token index:", DE_TEXT.vocab.stoi['<sos>'])
print("<eos> token index:", DE_TEXT.vocab.stoi['<eos>'])
print("<unk> token index:", DE_TEXT.vocab.stoi['<unk>'])


<pad> token index: 1
<sos> token index: 2
<eos> token index: 3
<unk> token index: 0


In [ ]:
print("<pad> token index:", EN_TEXT.vocab.stoi['<pad>'])
print("<sos> token index:", EN_TEXT.vocab.stoi['<sos>'])
print("<eos> token index:", EN_TEXT.vocab.stoi['<eos>'])
print("<unk> token index:", EN_TEXT.vocab.stoi['<unk>'])


<pad> token index: 1
<sos> token index: 2
<eos> token index: 3
<unk> token index: 0


In [ ]:
#class Embedder(nn.Module):
#  def __init__(self, vocab_size, d_model):
#    super().__init__()
#    self.embed = nn.Embedding(vocab_size, d_model, padding_idx = 0)
#  def forward(self, x):
#    try:
#      return self.embed(x)
#    except:
#      print(x)

In [ ]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        try:
            return self.embed(x)
        except IndexError as e:
            print("Caught an IndexError while embedding the input:")
            print(f"Input tensor shape: {x.shape}")
            print(f"Maximum token index in the input tensor: {torch.max(x)}")
            print(f"Vocabulary size: {self.embed.num_embeddings}")
            print(f"Embedding matrix shape: {self.embed.weight.shape}")
            raise e

In [ ]:
class Positional_Embedder(nn.Module):
  def __init__(self, d_model, max_seq_l=100):
    super().__init__()
    self.d_model = d_model

    pe = torch.zeros(max_seq_l, d_model)
    for pos in range(max_seq_l):
      for i in range(0, d_model, 2):
        pe[pos, i] = math.sin( pos / (10000 ** ( (2*i) / d_model)))
        pe[pos, i+1] = math.cos( pos / (10000 ** ( (2*(i+1)) / d_model)))

    pe = pe.unsqueeze(0)
    self.register_buffer('pe', pe)

  def forward(self, x):
    x = x * math.sqrt(self.d_model)
    seq_len = x.size(1)
    x = x + self.pe[:, :seq_len].detach().to(x.device)
    return x

In [ ]:
batch = next(iter(train_iter))
print(batch)


[torchtext.data.batch.Batch of size 32 from MULTI30K]
	[.src]:[torch.cuda.LongTensor of size 11x32 (GPU 0)]
	[.trg]:[torch.cuda.LongTensor of size 18x32 (GPU 0)]


In [ ]:
input_pad = EN_TEXT.vocab.stoi['<pad>']
target_pad = DE_TEXT.vocab.stoi['<pad>']

In [ ]:
def create_masks(src, trg):


    # creates mask with 0s wherever there is padding in the input
    src_mask = (src != input_pad).unsqueeze(1)

    if trg is not None:
        trg_mask = (trg != target_pad).unsqueeze(1)
        size = trg.size(1)  # get seq_len for matrix
        np_mask = nopeak_mask(size)
        trg_mask = trg_mask & np_mask

    else:
        trg_mask = None

    return src_mask, trg_mask

In [ ]:
def nopeak_mask(size):
    np_mask = np.triu(np.ones((1, size, size), dtype=np.uint8), k=1)
    np_mask = torch.tensor(np_mask == 0, dtype=torch.bool)
    return np_mask

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, heads, d_model, dropout=0.1):
    super().__init__()

    self.d_model = d_model
    self.d_k = d_model // heads
    self.h = heads

    self.q_lin = nn.Linear(d_model, d_model)
    self.k_lin = nn.Linear(d_model, d_model)
    self.v_lin = nn.Linear(d_model, d_model)
    self.drop = nn.Dropout(dropout)
    self.out = nn.Linear(d_model, d_model)

  def forward(self, q, k, v, mask=None):
    bs = q.size(0)
    q = self.q_lin(q).view(bs, -1, self.h, self.d_k)
    k = self.k_lin(k).view(bs, -1, self.h, self.d_k)
    v = self.v_lin(v).view(bs, -1, self.h, self.d_k)

    q = q.transpose(1, 2)
    k = k.transpose(1, 2)
    v = v.transpose(1, 2)

    scores = attention(q, k, v, self.d_k, mask, self.drop)
    concat = scores.transpose(1, 2).contiguous().view(bs, -1, self.d_model)
    output = self.out(concat)

    return output

In [ ]:
def attention(q, k, v, d_k, mask=None, drop=None):
  scores = torch.matmul(q, k.transpose(-2,-1))
  if mask is not None:
    mask = mask.unsqueeze(1)
    scores = scores.masked_fill(mask == 0, -1e9)
  scores = F.softmax(scores, dim=-1)

  if drop is not None:
    scores = drop(scores)

  output = torch.matmul(scores, v)
  return output

In [ ]:
class FeedForward(nn.Module):
  def __init__(self, d_model, d_ff=2048, dropout=0.1):
    super().__init__()
    self.lin_1 = nn.Linear(d_model, d_ff)
    self.drop = nn.Dropout(dropout)
    self.lin_2 = nn.Linear(d_ff, d_model)
  def forward(self, x):
    y = self.drop(F.relu(self.lin_1(x)))
    z = self.lin_2(y)
    return z

In [ ]:
class Norm(nn.Module):
  def __init__(self, d_model, eps = 1e-6):
    super().__init__()

    self.size = d_model
    self.alpha = nn.Parameter(torch.ones(self.size))
    self.bias = nn.Parameter(torch.zeros(self.size))
    self.eps = eps

  def forward(self, x):
    norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) / (x.std(dim=-1, keepdim=True) +self.eps) + self.bias
    return norm

In [ ]:
class EncoderLayer(nn.Module):
  def __init__(self, d_model, heads, drop=0.1):
    super().__init__()
    self.norm_mha = Norm(d_model)
    self.norm_ffn = Norm(d_model)
    self.attn = MultiHeadAttention(heads, d_model)
    self.ff = FeedForward(d_model)
    self.drop_mha = nn.Dropout(drop)
    self.drop_ffn = nn.Dropout(drop)

  def forward(self, x, mask):
    y = self.norm_mha(x)
    x = x + self.drop_mha(self.attn(y, y, y, mask))
    z = self.norm_ffn(x)
    op = x + self.drop_ffn(self.ff(z))
    return op

In [ ]:
class DecoderLayer(nn.Module):
  def __init__(self, d_model, heads, drop=0.1):
    super().__init__()
    self.norm_mmha = Norm(d_model)
    self.norm_mha = Norm(d_model)
    self.norm_ffn = Norm(d_model)
    self.drop_mmha = nn.Dropout(drop)
    self.drop_mha = nn.Dropout(drop)
    self.drop_ffn = nn.Dropout(drop)
    self.mmh_attn = MultiHeadAttention(heads, d_model)
    self.mh_attn = MultiHeadAttention(heads, d_model)
    self.ff = FeedForward(d_model)

  def forward(self, x, e_op, src_mask, trg_mask):
    y = self.norm_mmha(x)
    x = x + self.drop_mmha(self.mmh_attn(y, y, y, trg_mask))
    z = self.norm_mha(x)
    x = x + self.drop_mha(self.mh_attn(z, e_op, e_op, src_mask))
    w = self.norm_ffn(x)
    op = x + self.drop_ffn(self.ff(w))
    return op

In [ ]:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [ ]:
class Encoder(nn.Module):
  def __init__(self, vocab_size, d_model, N, heads):
    super().__init__()
    self.N = N
    self.embed = Embedder(vocab_size, d_model)
    self.pe = Positional_Embedder(d_model)
    self.layers = get_clones(EncoderLayer(d_model, heads), N)
    self.norm = Norm(d_model)
  def forward(self, src, mask):
    x = self.embed(src)
    x_emb = self.pe(x)
    for i in range(N):
      op = self.layers[i](x_emb, mask)
    return self.norm(op)

In [ ]:
class Decoder(nn.Module):
  def __init__(self, vocab_size, d_model, N, heads):
    super().__init__()
    self.N = N
    self.embed = Embedder(vocab_size, d_model)
    self.pe = Positional_Embedder(d_model)
    self.layers = get_clones(DecoderLayer(d_model, heads), N)
    self.norm = Norm(d_model)
  def forward(self, trg, e_outputs, src_mask, trg_mask):
    x = self.embed(trg)
    x_emb = self.pe(x)
    for i in range(self.N):
        op = self.layers[i](x_emb, e_outputs, src_mask, trg_mask)
    return self.norm(op)

In [ ]:
class Transformer(nn.Module):
  def __init__(self, src_vocab, trg_vocab, d_model, N, heads):
    super().__init__()
    self.encoder = Encoder(src_vocab, d_model, N, heads)
    self.decoder = Decoder(trg_vocab, d_model, N, heads)
    self.out = nn.Linear(d_model, trg_vocab)
  def forward(self, src, trg, src_mask, trg_mask):
    e_op = self.encoder(src, src_mask)
    d_op = self.decoder(trg, e_op, src_mask, trg_mask)
    op = self.out(d_op).to(src.device)
    return op

Hyperparameters (according to the paper)

In [ ]:
d_model = 512
heads = 8
N = 6
src_vocab = len(DE_TEXT.vocab)
trg_vocab = len(EN_TEXT.vocab)

In [ ]:
model = Transformer(src_vocab, trg_vocab, d_model, N, heads)

In [ ]:
model = model.to(device)

In [ ]:
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
# this code is very important! It initialises the parameters with a
# range of values that stops the signal fading or getting too big.
# See this blog for a mathematical explanation.
optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [ ]:
src = batch.src.transpose(0,1)
trg = batch.trg.transpose(0,1)
trg_input = trg[:, :-1]
targets = trg[:, 1:].contiguous().view(-1)
src_mask, trg_mask = create_masks(src, trg_input)
src_embedder = Embedder(vocab_size=src_vocab, d_model=d_model)
print(src_embedder.state_dict()['embed.weight'].size())

RuntimeError: ignored

In [ ]:
for i, batch in enumerate(train_iter):
  print(i)

In [ ]:
def train_model(epochs, print_every=100):

    model.train()
    total_loss = 0

    for epoch in range(epochs):

        for i, batch in enumerate(train_iter):
            src = batch.src.transpose(0,1)
            trg = batch.trg.transpose(0,1)

            trg_input = trg[:, :-1]
            targets = trg[:, 1:].contiguous().view(-1)
            src_mask, trg_mask = create_masks(src, trg_input)
            preds = model(src, trg_input, src_mask, trg_mask)
            optim.zero_grad()
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)), targets, ignore_index=target_pad)
            loss.backward()
            optim.step()

            total_loss += loss.item()
            if (i + 1) % print_every == 0:
                loss_avg = total_loss / print_every
                print("epoch %d, iter = %d, loss = %.3f" % (epoch + 1, i + 1, loss_avg))
                total_loss = 0

In [ ]:
train_model(10)

In [ ]:
def translate(model, src, max_len = 80, custom_sentence=False):
    model.eval()
    if custom_sentence == True:
        src = tokenize_en(src)
        sentence = torch.LongTensor([[EN_TEXT.vocab.stoi[tok] for tok in sentence]]).to(device)
        src_mask = (src != input_pad).unsqueeze(-2)
        e_outputs = model.encoder(src, src_mask)

        outputs = torch.zeros(max_len).type_as(src.data)
        outputs[0] = torch.LongTensor([DE_TEXT.vocab.stoi['<sos>']])
        for i in range(1, max_len):
          trg_mask = np.triu(np.ones((1, i, i), k=1)).astype('uint8')
          trg_mask = torch.tensor(trg_mask == 0).to(device)
          out = model.out(model.decoder(outputs[:i].unsqueeze(0), e_outputs, src_mask, trg_mask))
          out = F.softmax(out, dim=-1)
          val, ix = out[:, -1].data.topk(1)

          outputs[i] = ix[0][0]
          if ix[0][0] == DE_TEXT.vocab.stoi['<eos>']:
            break

        return ' '.join([DE_TEXT.vocab.itos[ix] for ix in outputs[:i]])